In [ ]:
import numpy as np

from Polyhedra import *
from Distances import *
from Algorithm import *

grid = np.array([
    [2, 0, 0, 0],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [1, 0, 0, 1]
], dtype = float)

grid1 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
], dtype = float)

grid2 = np.array([
    [1, -1, 0, 0],
    [0, 1, -1, 0],
    [0, 0, 1, -1],
    [0.5, 0.5, 0.5, 0.5]
], dtype = float)

grid3 = np.array([
    [1, 1, 0, 0],
    [1, -1, 0, 0],
    [0, 1, -1, 0],
    [0, 0, 1, -1]
], dtype = float)

In [ ]:
grid[0][0] = 2.15

vor4 = VoronoiPolyhedra(grid)
vor4.build()

det_dist, det_center, det_mat = find_optimal(range(49, 51), 2, grid, vor4)

